# 02 Neural Network Classification using Pytorch

In [1]:
import torch
import sklearn
from sklearn.datasets import make_circles
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from torch import nn  #neural networks
print(torch.__version__)

2.7.1+cu118


In [3]:
n_samples=1000
x,y=make_circles(n_samples,noise=0.03,random_state=42)
len(x),len(y)

(1000, 1000)

In [8]:
print(f"X shape: {x.shape}, y shape: {y.shape}\n ")
# Convert to tensors
print(f"first 5 samples of x:\n {x[:5]},\n \n first 5 samples of y:\n {y[:5]}")

X shape: (1000, 2), y shape: (1000,)
 
first 5 samples of x:
 [[ 0.75424625  0.23148074]
 [-0.75615888  0.15325888]
 [-0.81539193  0.17328203]
 [-0.39373073  0.69288277]
 [ 0.44220765 -0.89672343]],
 
 first 5 samples of y:
 [1 1 1 1 0]
